In [1]:
import random
import os
import numpy as np
import pandas as pd
import collections
import math

from pathlib import Path


from random import randrange
from shutil import copyfile
from numpy.linalg import norm
from math import sqrt

from sklearn.metrics import mean_squared_error


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
TRAIN_RATINGS_PATH = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-4/training_dataset.csv'
#TRAIN_RATINGS_PATH = '/content/drive/My Drive/3Sem/CS5683/Projects/Project 4/training_dataset.csv'
TEST_RATINGS_PATH = '/content/drive/My Drive/Coursework/CS 5683_Big_Data/Project-4/test_dataset.csv'
#TEST_RATINGS_PATH = '/content/drive/My Drive/3Sem/CS5683/Projects/Project 4/test_dataset.csv'

train_data = pd.read_csv(TRAIN_RATINGS_PATH,sep=",", engine="python",names=["user_id", "item_id", "rating"],usecols=[0, 1, 2], skiprows=1)
test_data = pd.read_csv(TEST_RATINGS_PATH,sep=",", engine="python",names=["user_id", "item_id", "rating"],usecols=[0, 1, 2], skiprows=1)

In [4]:
# ts.pivot_table(index='item_id',columns='user_id',values='rating')

In [5]:
#sparsity
x = train_data.pivot_table(index='user_id',columns='item_id',values='rating')
x.isna().sum().sum()/(x.shape[0]*x.shape[1])
sparsity =[]
for i in x.columns:
  sparsity.append(x[i].isnull().sum()/len(x[i]))
np.mean(sparsity)

0.9394941070444783

In [6]:
#test code
#utility matrix
umDf=tr.pivot_table(columns='item_id', index='user_id', values='rating')
um = umDf.values
um_adj= umDf.values
users= list(umDf.index)
movies = list(umDf.columns)
movie_mean = [0 for j in range(len(movies))]
global_mean= tr.rating.mean()
global_count= tr.shape[0]-tr.rating.isnull().sum()
# Computing user mean ratings for every user
user_mean = [0 for j in range(umDf.shape[1])]
for i in range(umDf.shape[0]):
    m = 0
    cnt = 0
    for j in range(len(movies)):
        if not math.isnan(um[i][j]):
            m = m + um[i][j]
            cnt += 1
    user_mean[i] = m / cnt
    for j in range(0, len(movies)):
        if not math.isnan(um[i][j]):
            um[i][j] = um[i][j]  # -self.user_mean[i]
            um_adj[i][j] = um_adj[i][j]  -user_mean[i]
        else:
            um[i][j] = 0
            um_adj[i][j] = 0


NameError: ignored

 Goal: Predict the rating of the movies by a user <br>
 Step1: Find similar movies which are similar to movie 1 in test set, using correlation <br>
 Step 2: predict the movie rating using weighted average estimate
 Step 3: 

In [ ]:
umDf = train_data.pivot_table(index='user_id', columns='item_id', values='rating')
user_mean = [0 for j in range(umDf.shape[0])]
um = umDf.values
um_adj= umDf.values
movies = list(umDf.columns)
movie_mean = [0 for j in range(len(movies))]
global_mean = 0
global_count = 0
for i in range(1, umDf.shape[0]):
  m = 0
  cnt = 0
  for j in range(len(movies)):
      if not math.isnan(um[i][j]):
          m = m + um[i][j]
          cnt += 1
  user_mean[i] = m / cnt
  global_mean += m
  global_count += cnt

for i in range(0, umDf.shape[0]):
    m = 0
    cnt = 0
    for j in range(len(movies)):
      if not math.isnan(um[i][j]):
        um[i][j]=um[i][j] 
        um_adj[i][j]=um_adj[i][j]- user_mean[i]
      else:
        um[i][j]=0
        um_adj[i][j]=0
global_mean=global_mean/global_count

for j in range(len(movies)):
  m = 0
  cnt = 0
  for i in range(umDf.shape[0]):
      if um[i][j] != 0:
          m = m + um[i][j]
          cnt = cnt + 1
  if cnt != 0:
      movie_mean[j] = m / cnt
# print(self.um[1][:],self.user_mean[1])
# self.um = np.array(self.um)

In [ ]:
def item_sim(um, i, j):
  return (np.dot(um[:,i],um[:,j])/(norm(um[:,i])*norm(um[:,j])))

In [ ]:
def top_sim_items(um,movies, user_index,i):
  ti = []
  for j in range(len(movies)):
    if j!=i and um[user_index][j] !=0:
      ti.append((item_sim(um,i,j),j))
  ti.sort(reverse=True)
  req=[]
  for z in ti:
    if z[0]>=0:
      req.append(z)
    else:
      continue
  return req[:15]


In [ ]:
x = top_sim_items(um,movies, 173,322)
x

[(0.20886896393370868, 986),
 (0.1127755836863854, 596),
 (0.10061498288968466, 400),
 (0.09788467182516174, 869),
 (0.09688296213762813, 385),
 (0.09579892527267574, 117),
 (0.09428124312249013, 1228),
 (0.0942364791242367, 368),
 (0.09269760110519278, 999),
 (0.09255104558422034, 387),
 (0.08716968114824926, 767),
 (0.08700876316862108, 903),
 (0.08213066137628926, 395),
 (0.08198830688177305, 363),
 (0.07955798774803177, 545)]

In [ ]:
def predict_rating(um,movies,user_mean, u, m):
  u_ind =users.index(u)
  m_ind = movies.index(m)
  ti = top_sim_items(um, movies, u_ind, m_ind)
  num, den = 0, 0
  for x in ti:
      num += x[0] * um[u_ind][x[1]]
      den += x[0]
  r = num / den
  r = r + user_mean[u_ind]
  return r


In [ ]:
predict_rating(um,movies,user_mean,655,823)

2.939891071858708

In [ ]:
x= np.arange(start=0,stop=test_df.shape[0],step=1)
x

array([   0,    1,    2, ..., 4980, 4981, 4982])

In [ ]:
count = 0
not_in_list =[]
for i in test_data.item_id.unique():
  if i not in train_data.item_id.unique():
    count+=1
    not_in_list.append(i)
print(count)
print(not_in_list)
#filtered test list
test_df = test_data[~test_data['item_id'].isin(not_in_list)]
test_df['new_ind']=x
test_df=test_df.set_index('new_ind')

print(test_df.index.max())
print(test_df.shape)
test_df

17
[1520, 1505, 1650, 830, 1453, 1494, 1340, 1565, 1559, 1447, 1352, 1636, 1572, 1576, 1603, 1633, 1457]
4982
(4983, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,user_id,item_id,rating
new_ind,,,
0,178,323,3
1,561,219,1
2,417,425,4
3,758,1292,1
4,269,357,5
...,...,...,...
4978,7,520,5
4979,592,292,1
4980,864,230,2


In [ ]:
test_df[test_df.index>=358]

,user_id,item_id,rating
new_ind,,,
358,363,290,3
359,234,432,4
360,194,770,4
361,881,205,4
362,181,880,1
...,...,...,...
4978,7,520,5
4979,592,292,1
4980,864,230,2


In [ ]:
# test_data[test_data.item_id==1520]
print(test_data.shape)
print(test_data[~test_data['item_id'].isin(not_in_list)].shape)

(5000, 3)
(4983, 3)


In [ ]:
#exception=[359,506,624,869,997,1091,1830,1921,1982,1992,2419]
exception=[]
u_id = test_df["user_id"]
m_id = test_df["item_id"]
Y = test_df["rating"].values
predictions = []
for i in range(len(u_id)):
  if i in exception :
    continue
  else:
    predictions.append(predict_rating(um,movies,user_mean,u_id[i], m_id[i]))
    if i % 200 == 0 and i != 0:
      print("RMSE at ", i, " :", sqrt(mean_squared_error(Y[: i + 1], predictions)))
#  if i ==359:
#    print("RMSE at ", i, " :", sqrt(mean_squared_error(Y[: i], predictions)))
#  else:
#    predictions.append(predict_rating(um,movies,user_mean,u_id[i], m_id[i]))
#    if i % 200 == 0 and i != 0:
#        print("RMSE at ", i, " :", sqrt(mean_squared_error(Y[: i + 1], predictions)))

RMSE at  200  : 0.9701734826323328
RMSE at  400  : 0.910862372100965
RMSE at  600  : 0.9052157783874717
RMSE at  800  : 0.9072129805469854
RMSE at  1000  : 0.9086335520918861
RMSE at  1200  : 0.8919986974443895
RMSE at  1400  : 0.8778112482390545
RMSE at  1600  : 0.8793617351368528
RMSE at  1800  : 0.8863923531203967
RMSE at  2000  : 0.8824670821363109
RMSE at  2200  : 0.8849376704938636
RMSE at  2400  : 0.884910494235758
RMSE at  2600  : 0.8869334146188956
RMSE at  2800  : 0.8901391996444993
RMSE at  3000  : 0.8966874791318212
RMSE at  3200  : 0.8938703347549881
RMSE at  3400  : 0.8961509893056457
RMSE at  3600  : 0.8931184454373099
RMSE at  3800  : 0.8921386664762808
RMSE at  4000  : 0.891425617406381
RMSE at  4200  : 0.8893970293404289
RMSE at  4400  : 0.8917407154975697
RMSE at  4600  : 0.8898795589369611
RMSE at  4800  : 0.8867477711801436


In [ ]:
print("RMSE ", sqrt(mean_squared_error(Y, predictions)))

RMSE  0.8885956581358273


## First version Code ends here

In [ ]:
#test
# approach with only test data
for i in ts.item_id.unique()[0:20]:
  ind = list(user_movie_rating.index).index(i)
  item_sim = []
# for i in range(user_movie_rating.shape[0]):
  sim_values=[]
  for j in range(user_movie_rating.shape[0]):
    numerator = np.dot(user_movie_rating.iloc[ind,:-2],user_movie_rating.iloc[j,:-2])
    denominator=(norm(user_movie_rating.iloc[ind,:-2])*norm(user_movie_rating.iloc[j,:-2]))
    if denominator !=0:
      value = numerator/denominator
    else:
      value=0
    sim_values.append(value)
  item_sim.append(sim_values)

In [ ]:
# item similarity
def item_sim(um, i, j):
    return (np.dot(um[1:, i], um[1:, j]) / (norm(um[1:, i]) * norm(um[1:, j])))

def top_sim_items(um, u, i):
    ti = []
    for j in range(0, len(movies)):
        if j != i and um[u][j] != 0:
            ti.append((item_sim(i, j), j))
    ti.sort(reverse=True)
    return ti[:15]

In [ ]:
def baseline(user_mean,movie_mean, u, m):
    return user_mean[u] + movie_mean[m] - global_mean

In [ ]:
def predict_rating(movies,user_mean,movie_mean,um, u, m):
  m = movies.index(m)
  ti = top_sim_items(um, u, m)
  num, den = 0, 0
  r=0
  for x in ti:
    num += x[0] * (um[u][x[1]] - baseline(user_mean,movie_mean,u, x[1]))
    den += x[0]
    if den ==0:
        r=0
    else:
      r = num/den
    r = r+baseline(user_mean,movie_mean,u,m)
  return r

In [ ]:
test_data

In [ ]:
m = movies.index(323)
ti = top_sim_items(um, 178, 323)

TypeError: ignored

In [ ]:
num, den = 0, 0
for x in ti:
  num += x[0] * (um[u][x[1]] - baseline(user_mean,movie_mean,u, x[1]))
  den += x[0]
  if den ==0:
    r=0
  else:
    r = num/den

In [ ]:
um = train_data.pivot_table(index='user_id',columns='item_id',values='rating').values
for i in range(0,um.shape[0]):
  for j in range(0, um.shape[1]):
    if not math.isnan(um[i][j]):
        um[i][j] = um[i][j]  #-user_mean[i]
    else:
        um[i][j] = 0

numpy.ndarray

In [ ]:
# print("Rating: ", predict_rating(178, 323))
u=178
m=323
print("Rating: ", predict_rating(movies,user_mean,movie_mean,um, u, m))

Rating:  0


In [ ]:
item_sim=[]

for i in ts.item_id.unique():
  ind = user_movie_rating.index(i)

for i in range(user_movie_rating.shape[0]):
  sim_values=[]
  for j in range(user_movie_rating.shape[0]):
    numerator = np.dot(user_movie_rating.iloc[i,:-2],user_movie_rating.iloc[j,:-2])
    denominator=(norm(user_movie_rating.iloc[i,:-2])*norm(user_movie_rating.iloc[i,:-2]))
    if denominator !=0:
      value = numerator/denominator
    else:
      value=0
    sim_values.append(value)
  item_sim.append(sim_values)

In [ ]:
train_data.shape

(95000, 3)